In [40]:
import pandas as pd
import numpy as np
import import_ipynb
from unidecode import unidecode
import glob
import csv
import warnings
from datetime import  datetime

import validItems

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.set_option('display.max_columns', 500)


In [41]:
obras = glob.glob(r"*.xlsx")
obras

['L01_AB - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx',
 'L03_PLAN-PROG-SEMANAL-OBRAS- 30-01-2023 a 05-02-2023.xlsx',
 'L06_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23_ARTESP_Rev00.xlsx',
 'L07_Cópia de Programação de Obras 30.01.2023 a 05.02.2023.xlsx',
 'L09_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23.xlsx',
 'L11_LOTE11_PLAN-PROG-SEMANAL-OBRAS- 30-01-23_a_05-02-23.xlsx',
 'L12_VO - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx',
 'L13_005_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23.xlsx',
 'L16_PLAN-PROG-SEMANAL-OBRAS-30-01-23 a 05-02-23_Final.xlsx',
 'L19_PLAN-PROG-SEMANAL-OBRAS-30-01 a 05-02.xlsx',
 'L20_SV - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx',
 'L21_PLAN-PROG-SEMANAL-OBRAS - 30-01-2023 a 05-02-2023.xlsx',
 'L22_Planilha -  Programacao de Obras e Servicos 30.01.2023 a 05.02.2023.xlsx',
 'L23_PLAN-PROG-SEMANAL-OBRAS-30-01-23 a 05-02-23.xlsx',
 'L24_RA - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx',
 'L25_ENG_PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx',
 'L25_TI_P

In [42]:
def treat_list(valid_counties):
    municipalities_treated = []
    for i in range(len(valid_counties)):
        valid_counties[i] = unidecode(valid_counties[i])
        valid_counties[i] = valid_counties[i].lower()
        municipalities_treated.append(valid_counties[i])
    return municipalities_treated


def check_highway(valid_highways, worksheet_highways):
    for i in range(len(worksheet_highways)):
        if not worksheet_highways[i] in valid_highways:
            wrong_list.append(f"Na planilha: {obra[0:3]} a rodovia esta errada! | a rodovia: {worksheet_highways[i]} nao existe!")
            
        
def check_codes(valid_codes, worksheet_codes):
    for i in range(len(worksheet_codes)):
        if not worksheet_codes[i] in valid_codes:
            wrong_list.append(f"Na planilha: {obra[0:3]} o cod de servico esta errado! | o codigo de servico: {worksheet_codes[i]} nao existe!")


def check_counties(valid_counties, worksheet_counties):
    for i in range(len(worksheet_counties)):
        if not unidecode(worksheet_counties[i].strip()) in valid_counties:
            wrong_list.append(f"Na planilha: {obra[0:3]} o municipio esta errado! | o municipio: {worksheet_counties[i].strip()} nao e do estado de Sao Paulo!")
            

def check_coordinates(worksheet, collumn_name: str):
    comma = ","
    coordinates_list = []
    for i in range(len(worksheet)):
        coordinates_list.append(worksheet[collumn_name][i])
    comma_list = [ a for a in coordinates_list if comma in str(a)]

    if len(comma_list) != 0:
        for i in range(len(comma_list)):
            wrong_list.append(f"Na planilha: {obra[0:3]} a coordenada esta errada! | esta com , no valor: {comma_list[i]}")

def check_senses(worksheet, valid_senses):  
    for i in range(len(worksheet)):
        if not worksheet[i].strip() in valid_senses:
            worksheet['SENTIDO'][i] = worksheet['SENTIDO'][i].strip()
            wrong_list.append(f"Na planilha: {obra[0:3]} o sentido esta errado! | o sentido: {worksheet[i]} não existe!")


def check_batch(worksheet_list, valid_batch):
    for i in range(len(worksheet_list)):
        if not worksheet_list[i] in valid_batch:
            wrong_list.append(f"Na planilha: {obra[0:3]} o lote esta errado! | o lote: {worksheet_list[i]} nao existe!")


def check_empty(worksheet, collumn_name):
    empty_list = []
    value_empty = []
    for i in range(len(worksheet)):
        if worksheet[collumn_name][i]:
            empty_list.append(worksheet[collumn_name][i])

    for i in range(len(empty_list)):
        if "nan" in str(empty_list[i]):
            value_empty.append(str(empty_list[i]).strip())
    if len(value_empty) > 0:
        wrong_list.append(f'Na planilha: {obra[0:3]}, na coluna {collumn_name}, existem {len(value_empty)} campos vazios!')
            
def check_km(worksheet, collumn_name: str):
    dot = "."
    comma = ","
    km_list = []
    for i in range(len(worksheet)):
        km_list.append(worksheet[collumn_name][i])
    dot_list = [a for a in km_list if dot in str(a)]
    comma_list = [ a for a in km_list if comma in str(a)]

    if len(comma_list) != 0 or len(dot_list) != 0:
        if len(comma_list) > 0:
            for i in range(len(comma_list)):
                wrong_list.append(f"Na planilha: {obra[0:3]} | Erro de KM, O KM {comma_list[i]} esta com ,")
        else:
            for i in range(len(dot_list)):
                wrong_list.append(f"Na planilha: {obra[0:3]} | Erro de KM, O KM {dot_list[i]} esta com .")
        return True
    else:
        return False

def check_dates(worksheet, collumn_name):
    invalid_dates = []
    dates = []
    interval = pd.date_range(start='2023-01-30', end='2023-02-05')

    for i in range(len(worksheet)):
        dates.append(worksheet[collumn_name][i])
        if not dates[i] in interval:
            invalid_dates.append(dates[i])

    if len(invalid_dates) > 0:
        wrong_list.append(f'Na planilha: {obra[0:3]}, na coluna {collumn_name}, existem {len(invalid_dates)} datas fora do periodo!')

def replacement(worksheet, collumn_name, to_alter, changed):
    worksheet[collumn_name] = worksheet[collumn_name].apply(lambda valor: str(valor).replace(to_alter, changed))

def concatenate_csv(first_file, second_file):
    csv_semana_passada = pd.read_csv(first_file, sep=',', low_memory=False, dtype= {'LOTE': object, 'CÓD. DO SERVIÇO': object})
    csv_semana_atual = pd.read_csv(second_file, sep=',', low_memory=False, dtype= {'LOTE': object, 'CÓD. DO SERVIÇO': object})
    obras_consolidada = pd.concat([csv_semana_passada, csv_semana_atual])
    obras_consolidada = obras_consolidada[['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO',
       'DETALHAMENTO DO SERVIÇO', 'KM INICIAL', 'LAT KM INICIAL',
       'LON KM INICIAL', 'KM FINAL', 'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO',
       'DATA INÍCIO', 'HORA INÍCIO', 'DATA FIM', 'HORA FIM', 'FI1', 'FI2',
       'FI3', 'FI4', 'FI5', 'FI6', 'FIAC', 'ALT.', 'FAIXAS INTERDITADAS',
       'EMPRESA RESPONSÁVEL', 'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
       'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
       'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']]
    obras_consolidada.to_csv("OBRAS.CSV", sep=',', index=False)


In [43]:
cabecalho = ['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO', 'DETALHAMENTO DO SERVIÇO',
       'KM INICIAL', 'LAT KM INICIAL', 'LON KM INICIAL', 'KM FINAL',
       'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO', 'DATA INÍCIO', 'HORA INÍCIO',
       'DATA FIM', 'HORA FIM', 'FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6',
       'FIAC', 'ALT.', 'EMPRESA RESPONSÁVEL',
       'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
       'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
       'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']

In [44]:
df_lista = []
wrong_list = []

for obra in obras:
    print("")
    print("planilha: ", obra)
    extensao = obra.split('.')[-1]
    print("extensao: ", extensao)
    df = pd.read_excel(obra, sheet_name="DADOS", 
        dtype={'DATA INÍCIO': object, 'DATA FIM': object, 'OBSERVAÇÕES GERAIS': object, 'DETALHAMENTO DO SERVIÇO': object, 'EMPRESA RESPONSÁVEL': object,
        'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA': object, 'APOIO PMRV': object, 'MUNICÍPIO': object, 'ART DO RESPONSÁVEL TÉCNICO': object},
        names=cabecalho, usecols=[x for x in range(30)])

    cod_planilha = []
    rodovia_planilha = []
    municipios_planilha = []
    sentidos = []
    lote = []
    for i in range(len(df)):
        cod_planilha.append(df["CÓD. DO SERVIÇO"][i])
        rodovia_planilha.append(df["RODOVIA"][i])
        municipios_planilha.append(unidecode(df["MUNICÍPIO"][i].lower()))
        sentidos.append(df['SENTIDO'][i])
        lote.append(df["LOTE"][i])

    check_batch(lote, validItems.lotes)
    check_highway(validItems.rodovias, rodovia_planilha)
    check_codes(validItems.codigo_servico, cod_planilha)
    check_km(df, 'KM INICIAL')
    check_coordinates(df,"LAT KM INICIAL")
    check_coordinates(df,"LON KM INICIAL")
    check_km(df, 'KM FINAL')
    check_dates(df, 'DATA FIM')
    check_dates(df, 'DATA INÍCIO')
    check_coordinates(df,"LAT KM FINAL")
    check_coordinates(df,"LON KM FINAL")
    check_senses(sentidos, validItems.sentido_validos)
    check_empty(df, "FI1")
    check_empty(df, "FI2")
    check_empty(df, "FI3")
    check_empty(df, "FI4")
    check_empty(df, "FI5")
    check_empty(df, "FI6")
    check_empty(df, "FIAC")
    check_empty(df, "ALT.")
    check_empty(df, "EMPRESA RESPONSÁVEL")
    check_empty(df, "RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA")
    check_empty(df, "ART DO RESPONSÁVEL TÉCNICO")
    check_empty(df, "NÍVEL DE SERVIÇO C/ INTERVENÇÃO")
    check_counties(treat_list(validItems.municipios), municipios_planilha)
    df_lista.append(df)


if len(wrong_list) > 0:
    print("Não podemos seguir por que a erros nas planilhas")
    for i in range(len(wrong_list)):
        print(wrong_list[i])
        arquivo_erros = open("Erros.txt", 'a', encoding='utf8')
        arquivo_erros.write(str(wrong_list[i]))
        arquivo_erros.write("\n")

decisao = input("Nas planilhas constam erros (Estão no Erros.txt), VOCÊ VERIFICOU OS ERROS? (S para continuar e gerar o csv // C se estiver usando somente para conferir a planilha // N para parar o programa)")


planilha:  L01_AB - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx
extensao:  xlsx

planilha:  L03_PLAN-PROG-SEMANAL-OBRAS- 30-01-2023 a 05-02-2023.xlsx
extensao:  xlsx

planilha:  L06_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23_ARTESP_Rev00.xlsx
extensao:  xlsx

planilha:  L07_Cópia de Programação de Obras 30.01.2023 a 05.02.2023.xlsx
extensao:  xlsx

planilha:  L09_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23.xlsx
extensao:  xlsx

planilha:  L11_LOTE11_PLAN-PROG-SEMANAL-OBRAS- 30-01-23_a_05-02-23.xlsx
extensao:  xlsx

planilha:  L12_VO - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx
extensao:  xlsx

planilha:  L13_005_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23.xlsx
extensao:  xlsx

planilha:  L16_PLAN-PROG-SEMANAL-OBRAS-30-01-23 a 05-02-23_Final.xlsx
extensao:  xlsx

planilha:  L19_PLAN-PROG-SEMANAL-OBRAS-30-01 a 05-02.xlsx
extensao:  xlsx

planilha:  L20_SV - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx
extensao:  xlsx

planilha:  L21_PLAN-PROG-SEMANAL-OBRAS - 30-01-2023 a 05-02-2023.

C:\Users\Thiago Mussolini\AppData\Local\Temp\ipykernel_10388\2472913890.py:92: UserWarning: Parsing '31/01/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  if not dates[i] in interval:
C:\Users\Thiago Mussolini\AppData\Local\Temp\ipykernel_10388\2472913890.py:92: UserWarning: Parsing '30/01/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  if not dates[i] in interval:


Não podemos seguir por que a erros nas planilhas
Na planilha: L01, na coluna EMPRESA RESPONSÁVEL, existem 2 campos vazios!
Na planilha: L03, na coluna DATA FIM, existem 10 datas fora do periodo!
Na planilha: L22, na coluna RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA, existem 83 campos vazios!
Na planilha: L25 o municipio esta errado! | o municipio: cidade tiradentes nao e do estado de Sao Paulo!
Na planilha: L25 o municipio esta errado! | o municipio: cidade tiradentes nao e do estado de Sao Paulo!
Na planilha: L25 o municipio esta errado! | o municipio: cidade tiradentes nao e do estado de Sao Paulo!
Na planilha: L25 o municipio esta errado! | o municipio: cidade tiradentes nao e do estado de Sao Paulo!
Na planilha: L25 o municipio esta errado! | o municipio: cidade tiradentes nao e do estado de Sao Paulo!
Na planilha: L25 o municipio esta errado! | o municipio: cidade tiradentes nao e do estado de Sao Paulo!
Na planilha: L25 o municipio esta errado! | o municipio: cidade tiradentes nao 

In [45]:
if decisao == "S":
       try:
              df_obras = pd.concat(df_lista, axis=0, ignore_index=True)
              data2 = pd.to_datetime(df_obras['DATA FIM'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
              df_obras['DATA FIM'] = pd.to_datetime(df_obras['DATA FIM'], errors='coerce', format='%d/%m/%Y')
              df_obras['DATA FIM'] = df_obras['DATA FIM'].fillna(data2)    
              data2 = pd.to_datetime(df_obras['DATA INÍCIO'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
              df_obras['DATA INÍCIO'] = pd.to_datetime(df_obras['DATA INÍCIO'], errors='coerce', format='%d/%m/%Y')
              df_obras['DATA INÍCIO'] = df_obras['DATA INÍCIO'].fillna(data2)
              replacement(df_obras, 'KM INICIAL','+', '')
              replacement(df_obras,'KM FINAL','+', '')
              replacement(df_obras,'KM INICIAL',',', '')
              replacement(df_obras, 'KM FINAL',',', '')
              df_obras['KM INICIAL'] = pd.to_numeric(df_obras['KM INICIAL'], errors='coerce')
              df_obras['KM FINAL'] = pd.to_numeric(df_obras['KM FINAL'], errors='coerce')
              df_obras['KM INICIAL'] = df_obras['KM INICIAL'] / 1000
              df_obras['KM FINAL'] = df_obras['KM FINAL'] / 1000
              df_obras['OBSERVAÇÕES GERAIS'] = df_obras['OBSERVAÇÕES GERAIS'].replace(r'\s+|\\n', ' ', regex=True)
              df_obras['DETALHAMENTO DO SERVIÇO'] = df_obras['DETALHAMENTO DO SERVIÇO'].replace(r'\s+|\\n', ' ', regex=True)
              df_obras['EMPRESA RESPONSÁVEL'] = df_obras['EMPRESA RESPONSÁVEL'].replace(r'\s+|\\n', ' ', regex=True)
              df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'] = df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'].replace(r'\s+|\\n', ' ', regex=True)
              df_obras['APOIO PMRV'] = df_obras['APOIO PMRV'].replace(r'\s+|\\n', ' ', regex=True)
              df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].replace(r'\s+|\\n', ' ', regex=True)
              df_obras = df_obras.iloc[:, :30]
              df_rodovia_nulo = df_obras['RODOVIA'].isnull()
              df= df[~df_rodovia_nulo]
              df_obras.reset_index(drop=True, inplace=True)
              df_obras["LOTE"] = df_obras["LOTE"].astype(int)
              df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
              replacement(df_obras, 'SENTIDO','OL','LO')
              replacement(df_obras, 'SENTIDO', 'IE','EI')
              replacement(df_obras,'SENTIDO', 'SN','NS')
              replacement(df_obras, 'SENTIDO', 'Leste','L')
              replacement(df_obras, 'SENTIDO', 'Norte','N')
              replacement(df_obras, 'SENTIDO', 'Oeste','O')
              replacement(df_obras, 'SENTIDO', 'Sul','S')
              replacement(df_obras, 'SENTIDO', 'Leste / Oeste','LO')
              replacement(df_obras, 'SENTIDO', 'Norte / Sul','NS')
              replacement(df_obras, 'SENTIDO', 'NN','N')
              replacement(df_obras, 'SENTIDO', 'SS','S')
              replacement(df_obras, 'SENTIDO', 'LO        ','LO')
              replacement(df_obras, 'SENTIDO', 'NS        ','NS')
              replacement(df_obras, 'SENTIDO', 'N         ','N')
              replacement(df_obras, 'SENTIDO', 'O         ','O')
              replacement(df_obras, 'SENTIDO', 'S         ','S')
              replacement(df_obras, 'MUNICÍPIO', ' -SP','')
              replacement(df_obras,'MUNICÍPIO', ' - SP','')
              df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: valor.split('/')[0])
              df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'] = df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'].replace(np.nan,'NÃO INFORMADO')
              df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].str.upper()
              df_obras['OBSERVAÇÕES GERAIS'] = df_obras['OBSERVAÇÕES GERAIS'].str.upper()
              df_obras['RODOVIA'] = df_obras['RODOVIA'].str.upper()
              df_obras['DETALHAMENTO DO SERVIÇO'] = df_obras['DETALHAMENTO DO SERVIÇO'].str.upper()
              df_obras['APOIO PMRV'] = df_obras['APOIO PMRV'].str.upper()
              df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'] = df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'].str.upper()
              df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'] = df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'].str.upper()
              df_obras['EMPRESA RESPONSÁVEL'] = df_obras['EMPRESA RESPONSÁVEL'].str.upper()
              df_obras['SENTIDO'] = df_obras['SENTIDO'].str.upper()
              df_obras['FI1'] = df_obras['FI1'].str.upper()
              df_obras['FI2'] = df_obras['FI2'].str.upper()
              df_obras['FI3'] = df_obras['FI3'].str.upper()
              df_obras['FI4'] = df_obras['FI4'].str.upper()
              df_obras['FI5'] = df_obras['FI5'].str.upper()
              df_obras['FI6'] = df_obras['FI6'].str.upper()
              df_obras['FIAC'] = df_obras['FIAC'].str.upper()
              replacement(df_obras, 'FI1', 'S','1')
              replacement(df_obras, 'FI2', 'S','2')
              replacement(df_obras, 'FI3', 'S','3')
              replacement(df_obras, 'FI4', 'S','4')
              replacement(df_obras, 'FI5', 'S','5')
              replacement(df_obras,'FI6', 'S','6')
              replacement(df_obras, 'FIAC', 'S','ACOS')
              df_obras['FAIXAS INTERDITADAS'] = df_obras['FI1'] + ',' + df_obras['FI2'] + ',' + df_obras['FI3'] + ','+ df_obras['FI4'] + ',' + df_obras['FI5'] + ',' + df_obras['FI6'] + ',' + df_obras['FIAC']
              replacement(df_obras, 'FAIXAS INTERDITADAS', 'N','NENHUMA')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '6NENHUMA','6')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '4NENHUMA','4')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '3NENHUMA','3')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '2NENHUMA','2')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '1NENHUMA','1')
              replacement(df_obras, 'FAIXAS INTERDITADAS', 'NENHUMA,','')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '1,NENHUMA','1')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '2,NENHUMA','2')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '3,NENHUMA','3')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '4,NENHUMA','4')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '5,NENHUMA','5')
              replacement(df_obras, 'FAIXAS INTERDITADAS', '6,NENHUMA','6')
              df_obras = df_obras[['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO', 'DETALHAMENTO DO SERVIÇO',
                     'KM INICIAL', 'LAT KM INICIAL', 'LON KM INICIAL', 'KM FINAL',
                     'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO', 'DATA INÍCIO', 'HORA INÍCIO',
                     'DATA FIM', 'HORA FIM', 'FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6',
                     'FIAC', 'ALT.', 'FAIXAS INTERDITADAS' , 'EMPRESA RESPONSÁVEL',
                     'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
                     'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
                     'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']]
              df_obras.to_csv("OBRAS_26_12_2022_a_01_01_2023.csv", encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC, index=False)
              concatenate_csv("OBRAS_26_12_2022_a_01_01_2023.csv","OBRAS_26_12_2022_a_01_01_2023.csv")
              print("CSV consolidade gerado com sucesso!!")
       except ValueError:
              print("Não passou pelas validações corriga para tentar novamente")
elif decisao == "C":
       print("Terminado de conferir, verifique se não constam erros no arquivo erros.txt")
else:
       print("Solicite correção aos erros, e em seguida repita o processo!")

C:\Users\Thiago Mussolini\AppData\Local\Temp\ipykernel_10388\301548651.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df= df[~df_rodovia_nulo]


CSV consolidade gerado com sucesso!!
